In [ ]:
# Developer Mode -> Develop -> qwiklabs-ecommerce

##Task 1. Enable the Cloud Document AI API and copy lab source files

In [ ]:
# -> API & Services -> Library -> Cloud Document AI API -> Enable

In [ ]:
  mkdir ./document-ai-challenge
  gsutil -m cp -r gs://spls/gsp367/* \
    ~/document-ai-challenge/

##Task 2. Create a form processor

In [ ]:
# Document AI -> Explore Processors -> Form Parser: Create Processor -> Processor Name:	form-processor

##Task 3. Create Google Cloud resources

In [ ]:
export PROJECT_ID=$(gcloud config get-value core/project)
export BUCKET_LOCATION=us-east4

In [ ]:
gsutil mb -c standard -l ${BUCKET_LOCATION} -b on \
 gs://${PROJECT_ID}-input-invoices
gsutil mb -c standard -l ${BUCKET_LOCATION} -b on \
 gs://${PROJECT_ID}-output-invoices
gsutil mb -c standard -l ${BUCKET_LOCATION} -b on \
 gs://${PROJECT_ID}-archived-invoices

In [ ]:
bq --location="US" mk  -d \
   --description "Form Parser Results" \
   ${PROJECT_ID}:invoice_parser_results

In [ ]:
cd ~/documentai-pipeline-demo/scripts/table-schema/

In [ ]:
bq mk --table invoice_parser_results.doc_ai_extracted_entities\doc_ai_extracted_entities.json
bq mk --table invoice_parser_results.geocode_details\geocode_details.json

##Task 4. Deploy the document processing Cloud Run functions

In [ ]:
cd ~/document-ai-challenge/scripts

In [ ]:
PROJECT_ID=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects list --filter="project_id:$PROJECT_ID" --format='value(project_number)')

SERVICE_ACCOUNT=$(gcloud storage service-agent --project=$PROJECT_ID)

gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member serviceAccount:$SERVICE_ACCOUNT \
  --role roles/pubsub.publisher

In [ ]:
  export CLOUD_FUNCTION_LOCATION=us-east4
  gcloud functions deploy process-invoices \
  --gen2 \
  --region=${CLOUD_FUNCTION_LOCATION} \
  --entry-point=process_invoice \
  --runtime=python39 \
  --service-account=${PROJECT_ID}@appspot.gserviceaccount.com \
  --source=cloud-functions/process-invoices \
  --timeout=400 \
  --env-vars-file=cloud-functions/process-invoices/.env.yaml \
  --trigger-resource=gs://${PROJECT_ID}-input-invoices \
  --trigger-event=google.storage.object.finalize\
  --service-account $PROJECT_NUMBER-compute@developer.gserviceaccount.com \
  --allow-unauthenticated

##Task 5. Test and validate the end-to-end solution

In [ ]:
# Cloud Functions -> process-invoices -> Edit & Deploy -> Variables & Secrets -> Processor ID ... [-> Document AI] -> Deploy
# -> Source -> Edit Source -> .env.yaml -> Processor ID ... [-> Document AI] -> Save & Redeploy

In [ ]:
gsutil -m cp -r gs://cloud-training/gsp367/* \
 ~/document-ai-challenge/invoices gs://${PROJECT_ID}-input-invoices/